In [2]:
#Generación Archivo
#importar librerias
import csv
import numpy as np
import pandas as pd
import random as rd
import string as st
# from google.colab import drive
from datetime import datetime
from datetime import timedelta
import google.cloud.storage as storage
import os

In [8]:
#mount drive
# from google.colab import drive
# drive.mount('/content/drive')

# path = '/content/drive/Shareddrives/Infra_Grupo_Sarasa/repositorio/data/TP_Final_Especificacion_1.xlsx'
path = '../data/TP_Final_Especificacion_1.xlsx'
path_coord ='../data/ar.csv'
path_destino = 'g1_distribuidor_'

credentials = '../usm-infra-grupo1-42c8c1c4178a.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials
storage_client = storage.Client()

In [9]:
# Leer el archivo de especificaciones
df_sheets_desc = pd.read_excel(path, sheet_name=None)
df_cond_vta = df_sheets_desc['Condicion_Venta']
df_unidades = df_sheets_desc['Unidades']
df_tipo_negocio = df_sheets_desc['Tipo_Negocio']
df_cadena = df_sheets_desc['Cadena']
df_dias_visita = df_sheets_desc['Dias_Visita']
df_productos = df_sheets_desc['Productos']
df_cond_venta = df_sheets_desc['Condicion_Venta']
df_localidades = df_sheets_desc['Prov_Localidades']
df_estado = df_sheets_desc['Estado_cliente']
df_dias_visita = df_sheets_desc['Dias_Visita']
coordenadas = pd.read_csv(path_coord)

In [10]:
def generar_codigos_sku(productos):
  codigos_sku = {}
  for producto in productos:
    caracteres = st.ascii_uppercase + st.digits
    sku_codigo = ''.join(rd.choice(caracteres) for _ in range(10))
    codigos_sku[producto] = sku_codigo

  return codigos_sku

def generate_fecha_cierre(fecha_cierre: datetime) -> str:
  """
  Genera una fecha de cierre de comprobante aleatoria entre la fecha de cierre
  de comprobante y la fecha actual.
  """
  fech_cie_com = (fecha_cierre - timedelta(hours=rd.randint(0, 23), minutes=rd.randint(0, 59), seconds=rd.randint(0, 59))) \
          .strftime('%Y-%m-%d %H:%M:%S')
  return fech_cie_com

In [ ]:
#generar SKUs a partir de la lista de productos
codigos_sku = generar_codigos_sku(df_productos['nombre'])

#nombre de las columnas de los archivos cliente,venta,stock
columns_v = df_sheets_desc['Desc_Ventas']['Campo']
columns_s = df_sheets_desc['Desc_Stock']['Campo']
columns_m = df_sheets_desc['Desc_Clientes']['Campo']
columns_d = df_sheets_desc['Desc_Deuda']['Campo']

columns_m=list(columns_m)
columns_m.remove('condicion_venta')
columns_m.remove('deuda_vencida')



#cantidad de días desde la fecha actual hacia el pasado que generamos datos
cant_dias = 2

#fecha actual
fecha_actual = datetime.now()

#cantidad de distribuidores
cant_dist = 2

#cantidad de clientes
cant_clientes = 4

#cantidad sucursales
cant_sucursales = 5

for distribuidor in range(1,cant_dist):
  codigo_sucursal_base = (distribuidor * 100)  # Código base para el distribuidor


  #sucursales del distribuidor
  #el dist1 va a tener sucursales de la 26 a 50
  #el dist2 va a tener sucursales de la 51 a 75
  sucursales_dist_total = [(codigo_sucursal_base + rd.randint(1, 99)) for _ in range(cant_sucursales)]
  #clientes de la sucursal del distribuidor
  cod_clientes = [(str(codigo_sucursal_base) + str(rd.randint(10, 99))) for _ in range(cant_clientes)]

  #Provincia y ciudades
  provincias = [df_localidades.columns.values[distribuidor%len(df_localidades.columns.values)] for _ in range(cant_clientes)]
  ciudades_aleatorias = []
  for _ in range(cant_clientes):
    for p in provincias:
        ciudades_provincia = df_localidades[p].dropna().tolist()
        ciudad_aleatoria = rd.choice(ciudades_provincia)
        ciudades_aleatorias.append(ciudad_aleatoria)

  coordenada_latitud = []
  coordenada_longitud = []
  for ciudad in ciudades_aleatorias:
      coordenada_latitud.append(list(coordenadas[coordenadas['city'] == ciudad]['lat'])[0])
      coordenada_longitud.append(list(coordenadas[coordenadas['city'] == ciudad]['lng'])[0])

  #Estado
  estado = [(rd.choices(df_estado['nombre'], weights=df_estado['probabilidades'], k=1)[0]) for _ in range(cant_clientes)]

  #Nombre_ cliente
  nombre_cliente = [(f'Cliente_{_}') for _ in range(cant_clientes)]

  #CUIT
  cuit = [(rd.randint(10000000, 99999999) ) for _ in range(cant_clientes)]

  #Razon Social
  razon_social = [(f"Empresa_{_}") for _ in range(cant_clientes)]

  #Direccion
  direccion = [(f"Dirección_{_}") for _ in range(cant_clientes)]

  #Dias de visita
  dias_visita = [(rd.choice(df_dias_visita['codigo_dia'])) for _ in range(cant_clientes)]

  #Telefono
  telefono = [(f"{rd.randint(100, 999)}-{rd.randint(100, 999)}-{rd.randint(1000, 9999)}") for _ in range(cant_clientes)]

  #Fecha de alta y de baja
  fecha_alta = [('2023-01-01') for _ in range(cant_clientes)]  # Fecha de alta corregir !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
  fecha_baja = [(fech_cierre + timedelta(days=rd.randint(cant_dias,365))).strftime('%Y-%m-%d') if estado== 'I'else np.nan]  # No hay fecha de baja corregir!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



  #Tipo de negocio
  tipo_negocio = [(rd.choice(df_tipo_negocio['nombre'])) for _ in range(cant_clientes)]

  sucursales_clientes=rd.choices(sucursales_dist_total,k=cant_clientes)

  clientes_df = pd.DataFrame(data = [sucursales_clientes,cod_clientes,ciudades_aleatorias,provincias,estado,nombre_cliente,cuit,razon_social,direccion,dias_visita,telefono,
                                  fecha_alta,fecha_baja,coordenada_latitud,coordenada_longitud,tipo_negocio],
                          columns= columns_m)



  for d in range(0,cant_dias):

    #fecha de cierre
    fech_cierre = fecha_actual - timedelta(days=d)
    fech_cierre_string = fech_cierre.strftime('%Y-%m-%d')
    fech_cie_com_data = [ (fech_cierre - timedelta(hours=rd.randint(0, 23), minutes=rd.randint(0, 59), seconds=rd.randint(0, 59))).strftime('%Y-%m-%d %H:%M:%S') for _ in range(cant_clientes)]


    #generamos los datos de venta unidades
    vta_unidades_data = [(rd.randint(0,100)) for _ in range(cant_clientes)]

    #generamos los datos de venta en importes
    vta_importe_data = [round((rd.uniform(100,1000)),2) for _ in range(cant_clientes)]

    #generamos la condicion de venta
    cond_vta_data = [ rd.choice(df_cond_vta['codigo_condicion']) for _ in range(cant_clientes)]


    clientes_tanda = pd.concat([clientes_df.iloc[rd.randint(0,cant_clientes)] for cliente in range(cant_clientes)])

    sucursales_dist = list(clientes_tanda['codigo_sucursal'])


    #ARCHIVO STOCK
    #Unidad
    unidad = [(rd.choice(df_unidades['codigo_unidad'])) for _ in range(cant_clientes)]
    #Stock
    stock = [(rd.randint(100, 500) if unidad == "UNI" else rd.randint(1, 100)) for _ in range(cant_clientes)]
    #Producto
    producto=[]
    producto1=[]
    producto1 = list(codigos_sku.keys())
    producto1.extend(producto1)
    producto=producto1
    #SKU
    sku_codigo=[]
    sku_codigo1=[]
    sku_codigo1 = list(codigos_sku.values())
    sku_codigo1.extend(sku_codigo1)
    sku_codigo = sku_codigo1

    #DEUDA
    deuda_vencida = [(round(rd.uniform(0, 1000), 2)) for _ in range(cant_clientes)]
    deuda_total = [(round(rd.uniform(0, 10000), 2)) for _ in range(cant_clientes)]

    clientes_tanda['deuda_vencida']=deuda_vencida
    clientes_tanda['condicion_venta']=cond_vta_data
    clientes=clientes_df['codigo_cliente']
    #data frame con data
    venta_clientes_data = list(zip(sucursales_dist,clientes,fech_cie_com_data,sku_codigo,vta_unidades_data,vta_importe_data,cond_vta_data))
    stock_data = list(zip(sucursales_dist_total, fech_cie_com_data, sku_codigo, producto, stock, unidad))
    #clientes_data = list(zip(sucursales_dist, clientes, ciudades_aleatorias, provincias, estado, nombre_cliente, cuit, razon_social, direccion, dias_visita, telefono, fecha_alta, fecha_baja, coordenada_latitud, coordenada_longitud, cond_vta_data, deuda_vencida, tipo_negocio))
    deuda_data = list(zip(sucursales_dist,clientes, fech_cie_com_data,deuda_vencida,deuda_total ))


    df_vta_cli = pd.DataFrame(venta_clientes_data,columns=columns_v)
    df_stock = pd.DataFrame(stock_data,columns=columns_s)
    #df_clientes = pd.DataFrame(clientes_data,columns=columns_m)
    df_deuda =  pd.DataFrame(deuda_data,columns=columns_d)

    #upload file
    carpeta_destino =  path_destino + str(distribuidor)
    bucket = storage_client.bucket(carpeta_destino)

    if not bucket.exists():
        bucket = storage_client.create_bucket(carpeta_destino)
    
    archivo_destino_v = f'venta/{fech_cierre_string}.csv'
    archivo_destino_s = f'stock/{fech_cierre_string}.csv'
    archivo_destino_m = f'cliente/{fech_cierre_string}.csv'
    archivo_destino_d = f'deuda/{fech_cierre_string}.csv'

    # blob = bucket.blob(archivo_destino_v)
    # blob.upload_from_string(df_vta_cli.to_csv(index=False), 'text/csv')

    # blob = bucket.blob(archivo_destino_s)
    # blob.upload_from_string(df_stock.to_csv(index=False), 'text/csv')

    # blob = bucket.blob(archivo_destino_m)
    # blob.upload_from_string(clientes_tanda.to_csv(index=False), 'text/csv')

    # blob = bucket.blob(archivo_destino_d)
    # blob.upload_from_string(df_deuda.to_csv(index=False), 'text/csv')


In [17]:
df_vta_cli.head()

Campo,codigo_sucursal,codigo_cliente,fecha_cierre_comercial,SKU_codigo,venta_unidades,venta_importe,condicion_venta
0,Córdoba,143,2023-11-02 05:31:52,GVW60CECJ0,57,125.29,EFE
1,10099,10088,2023-11-02 09:53:29,S03ZG7EXV2,14,335.72,CEX
2,10099,San Francisco,2023-11-01 18:49:58,CA27DTKAWZ,60,237.46,CEX
3,136,Córdoba,2023-11-02 15:44:13,4FXQ47Z88F,45,868.79,EFE


In [18]:
df_stock.head()

Campo,codigo_sucursal,fecha_cierre_comercial,SKU_codigo,SKU_descripcion,stock_unidades,unidad
0,175,2023-11-02 05:31:52,GVW60CECJ0,Leche,79,PAQ
1,128,2023-11-02 09:53:29,S03ZG7EXV2,Pan,21,UNI
2,143,2023-11-01 18:49:58,CA27DTKAWZ,Arroz,28,PAQ
3,136,2023-11-02 15:44:13,4FXQ47Z88F,Azúcar,91,UNI


In [19]:
clientes_tanda.head()

codigo_sucursal    Córdoba
codigo_cliente     Córdoba
ciudad             Córdoba
provincia          Córdoba
estado                None
dtype: object

In [20]:
df_deuda.head()

Campo,codigo_sucursal,codigo_cliente,fecha_cierre_comercial,deuda_vencida,deuda_total
0,Córdoba,143,2023-11-02 05:31:52,158.62,7437.39
1,10099,10088,2023-11-02 09:53:29,783.09,6952.89
2,10099,San Francisco,2023-11-01 18:49:58,111.12,3605.70
3,136,Córdoba,2023-11-02 15:44:13,274.13,9061.36
